In [1]:
import requests
from requests_oauthlib import OAuth1Session
from pymongo import MongoClient
import MySQLdb
import tweepy
import time
from datetime import datetime
import sys
import os

In [3]:
api_key = os.environ.get('TWITTER_API_KEY')
api_secret = os.environ.get('TWITTER_API_SECRET')
access_token = os.environ.get('TWITTER_ACCESS_TOKEN')
access_secret = os.environ.get('TWITTER_ACCESS_SECRET')

stock_num = 0

url = 'https://stream.twitter.com/1.1/statuses/filter.json'

In [5]:
twitter = OAuth1Session(api_key, api_secret, access_token, access_secret)

In [12]:
conn = MySQLdb.connect(\
                       host='localhost', \
                       port=3306, \
                       user='data_writer', \
                       passwd=os.environ.get('MYSQL_PASSRWORD') , \
                       db='twitter', \
                       charset='utf8mb4')
c = conn.cursor()

In [13]:
c.execute('''
  create table if not exists raw_data (
    id integer not null auto_increment,
    twitter_id integer,
    screen_name varchar(255),
    name varchar(255),
    user_location varchar(100),
    followers_cnt integer,
    listed_cnt integer,
    created_at timestamp,
    latitude double,
    longitude double,
    place_full_name varchar(255),
    place_name varchar(255),
    place_type varchar(255),
    text varchar(255),
    primary key (id)
  )
''')
conn.commit()

In [2]:
conn = MySQLdb.connect(\
                       host='localhost', \
                       port=3306, \
                       user='data_writer', \
                       passwd= os.environ.get('MYSQL_PASSWORD') ,\
                       db='yelp', \
                       charset='utf8')
c = conn.cursor()

In [3]:
c.execute('select count(*) from raw_data where business_id = "hogege"')
conn.commit()

In [6]:
res = c.fetchall()
res

()

In [8]:
if res:
    print('None')
else:
    print('kore')

kore


In [6]:
#r = requests.post(url, auth=auth, stream=True, data={"locations":"139.0,39,140.0, 39.5"}
response = twitter.get(url)

In [ ]:
for i, line in enumerate(r.iter_lines()):
    if i == 0:
        print(line)

In [8]:
response.json

<bound method Response.json of <Response [406]>>

In [9]:
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)

In [10]:
api = tweepy.API(auth)
public_tweets = api.home_timeline()

In [ ]:
public_tweets

In [111]:
client = MongoClient('localhost', 27017)
db = client.tweet_test

In [132]:
class MyStreamListerner(tweepy.StreamListener):
    
    def on_status(self, status):
        global stock_num
        if status.coordinates:
            stock_num += 1
            tweet = {}
            try:
                # print(status.retweet)
                tweet['id'] = status.user.id
                tweet['screen_name'] = status.user.screen_name
                tweet['name'] = status.user.name
                tweet['user_location'] = status.user.location
                tweet['followers_cnt'] = status.user.followers_count
                tweet['listed_cnt'] = status.user.listed_count
                tweet['created_at'] = status.created_at
                tweet['latitude'] = status.coordinates['coordinates'][1]
                tweet['longitude'] = status.coordinates['coordinates'][0]
                tweet['place_full_name'] = status.place.full_name
                tweet['place_name'] = status.place.name
                tweet['place_type'] = status.place.place_type
                tweet['text'] = status.text
                db.sample.insert_one(tweet)
                
                if stock_num % 1000.0 == 0.0:
                    print("STOCK:" + str(stock_num), datetime.now())
            except:
                stock_num -= 1
                try:
                    print('id:{} loss'.format(status.user.id), datetime.now())
                except:
                    print('Number:{} loss'.format(stock_num), datetime.now())
            if stock_num == 100000:
                print('get {} geo-tweet'.format(stock_num), datetime.now())
                sys.exit()
            return True
    
    def on_error(self, status_code):
        print('error:' + str(status_code))
        return True

In [17]:
class MySQLStreamListerner(tweepy.StreamListener):
    
    def on_status(self, status):
        global stock_num
        if status.coordinates:
            stock_num += 1
            sql_insert = '''insert into raw_data ( \ 
                twitter_id,\
                screen_name,\
                name,\
                user_location,\
                followers_cnt,\
                listed_cnt,\
                created_at,\
                latitude,\
                longitude,\
                place_full_name,\
                place_name,\
                place_type,\
                text)\
            values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
            insert_item = (
                status.user.id,
                status.user.screen_name,
                status.user.name,
                status.user.location,
                status.user.followers_count,
                status.user.listed_count,
                status.created_at,
                status.coordinates['coordinates'][1],
                status.coordinates['coordinates'][0],
                status.place.full_name,
                status.place.name,
                status.place.place_type,
                status.text,
            )
            c.execute(sql_insert, insert_item)
            conn.commit()
            try:
                #c.execute(sql_insert, insert_item)
                if stock_num % 1000.0 == 0.0:
                    print("STOCK:" + str(stock_num), datetime.now())
            except:
                stock_num -= 1
                try:
                    print('id:{} loss'.format(status.user.id), datetime.now())
                except:
                    print('Number:{} loss'.format(stock_num), datetime.now())
            if stock_num == 100000:
                print('get {} geo-tweet'.format(stock_num), datetime.now())
                sys.exit()
            return True
    
    def on_error(self, status_code):
        print('error:' + str(status_code))
        return True

In [18]:
stock_num = 0
stream = tweepy.Stream(auth, MySQLStreamListerner())
while True:
    try:
        stream.filter(languages=['ja'], locations=[128.416338,30.443678,146.113366,44.982576])
    except SystemExit:
        import traceback
        traceback.print_exc()
        break
    except:
        print('error occuerd. reconnection start in 60sec', datetime.now())
        time.sleep(60)

error occuerd. reconnection start in 60sec 2017-12-11 21:42:01.902757


KeyboardInterrupt: 

In [9]:
cate = {}

In [13]:
cate['hoge_hoge'] = "hoge"
cate

{'hoge_hoge': 'hoge'}

In [15]:
cate['hoge_hoge'] = cate['hoge_hoge'] + cate['hoge_hoge']

In [16]:
cate['hoge_hoge']

'hogehoge'

In [17]:
for i in range(10):
    cate['fuge'] += cate['hoge_hoge']

KeyError: 'fuge'